In [1]:
### import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Function to fetch historical stock returns and market index returns
def fetch_stock_and_market_returns(stock_ticker, market_index):
    stock_data = yf.download(stock_ticker, start='2020-01-01', end='2024-01-01')
    market_data = yf.download(market_index, start='2020-01-01', end='2024-01-01')
    stock_returns = stock_data['Adj Close'].pct_change().dropna()
    market_returns = market_data['Adj Close'].pct_change().dropna()
    return stock_returns, market_returns

# Calculate beta using linear regression
def calculate_beta(stock_returns, market_returns):
    model = LinearRegression()
    model.fit(market_returns.values.reshape(-1, 1), stock_returns.values)
    beta = model.coef_[0]
    return beta

def get_input_data():
            
    company_name = input("Enter the company name (ticker): ")
    market_index = input("Enter the market index (ticker) for the home country: ")
    country = input("Enter the country of operation: ")
    total_revenue = float(input("Enter total revenue of firm: "))
    COGS = float(input("Enter COGS: "))
    gross_profit = total_revenue - COGS
    SGnA = float(input("Enter SG&A expenses: "))
    RnD = float(input("Enter R&D expenses: "))
    O_p_e = float(input("Enter Other Operating Expenses: "))
    tax_rate = float(input("Enter the tax rate (in %): ")) / 100
    operating_expenses = SGnA+RnD+O_p_e
    operating_profit = gross_profit-operating_expenses
    after_tax_operating_profit = operating_profit*(1-tax_rate)
    total_debt = float(input("Enter total debt(short term + long term): "))
    cap_ex = float(input("Enter Capital Expenditure: "))
    DnA = float(input("Enter Depreciation and Amortization: "))
    current_assets = float(input("Enter current assets: "))
    current_liabilities = float(input("Enter current liabilities: "))
    cash = float(input("Enter current cash balance: "))
    non_cash_change_working_capital = (current_assets-cash)-current_liabilities
    shares_issued = float(input("Enter new shares issued this year: "))
    shares_repurchased = float(input("Enter shares repurchased this year: "))
    debt = float(input("Enter the market value of debt: "))
    equity = float(input("Enter the market value of equity: "))
    domestic_revenue_percentage = float(input("Enter the percentage of domestic revenue for the company: "))
    sector_domestic_revenue_percentage = float(input("Enter the percentage of domestic revenue for the average company in the same sector: "))
    implied_equity_risk_premium_sp500 = float(input("Enter the implied equity risk premium for S&P 500 (in %): "))
    bond_market_stddev = float(input("Enter the standard deviation of the bond market index (in %): ")) / 100
    company_default_spread = float(input("Enter the company default spread risk based on rating: "))
    if company_default_spread == 'help':
            ebit = float(input("Enter the company's EBIT: "))
            interest_expenses = float(input("Enter the company's Interest Expenses: "))
            company_default_spread, RATING = calculate_default_spread(ebit, interest_expenses)
            print(RATING, company_default_spread)
    else:
        try:
            company_default_spread = float(company_default_spread)
        except ValueError:
            print("Invalid input. Please enter a numerical value or type 'help'.")

    # Risk-free rate calculation inputs
    longest_gov_bond_rate = float(input("Enter the longest term government bond return rate (in %): "))
    default_spread = float(input(f"Enter the default spread for {country} (in %): "))
    terminal_gr = float(input("Enter Terminal Growth Rate (Use 3% for large, 4% for mid, 5% for small company): "))

    # Cost of debt calculation inputs
    revenue_country = domestic_revenue_percentage / 100
    other_revenue_country = 0
    other_default_spread = 0  # Initialize other_default_spread
    if revenue_country < 1:
        while revenue_country < 1:
            other_country = input("Enter the name of the second country: ")
            other_revenue_percentage = float(input(f"Enter the percentage of revenue from {other_country}: ")) / 100
            other_default_spread = float(input(f"Enter the default spread for {other_country} (in %): ")) / 100
            other_revenue_country += other_revenue_percentage
            revenue_country += other_revenue_percentage
            if revenue_country >= 1:
                break

    return (shares_issued, shares_repurchased, total_debt, terminal_gr, non_cash_change_working_capital, DnA, cap_ex, after_tax_operating_profit, company_name, market_index, domestic_revenue_percentage, sector_domestic_revenue_percentage, debt, equity, 
            implied_equity_risk_premium_sp500, bond_market_stddev, tax_rate, longest_gov_bond_rate, default_spread, other_revenue_country, other_default_spread, company_default_spread)

def calculate_default_spread(ebit, interest_expenses):
    interest_coverage_ratio = ebit / interest_expenses
    if interest_coverage_ratio > 12.5:
        company_default_spread = 0.59
        RATING = "AAA"
    elif interest_coverage_ratio > 9.5:
        company_default_spread = 0.7
        RATING = "AA"
    elif interest_coverage_ratio > 7.5:
        company_default_spread = 0.92
        RATING = "A"
    elif interest_coverage_ratio > 6:
        company_default_spread = 1.07
        RATING = "A2"
    elif interest_coverage_ratio > 4.5:
        company_default_spread = 1.21
        RATING = "A3"
    elif interest_coverage_ratio > 4:
        company_default_spread = 1.47
        RATING = "BBB"
    elif interest_coverage_ratio > 3.5:
        company_default_spread = 1.74
        RATING = "BB+"
    elif interest_coverage_ratio > 3:
        company_default_spread = 2.21
        RATING = "BB"
    elif interest_coverage_ratio > 2.5:
        company_default_spread = 3.14
        RATING = "B"
    elif interest_coverage_ratio > 2:
        company_default_spread = 3.61
        RATING = "B2"
    elif interest_coverage_ratio > 1.5:
        company_default_spread = 5.24
        RATING = "B3"
    elif interest_coverage_ratio > 1.25:
        company_default_spread = 8.51
        RATING = "CCC"
    elif interest_coverage_ratio > 0.8:
        company_default_spread = 11.78
        RATING = "CC"
    elif interest_coverage_ratio > 0.5:
        company_default_spread = 17
        RATING = "C"
    else:
        company_default_spread = 20
        RATING = "D"
        
    return (company_default_spread, RATING)

def dcf_calculate():
    (shares_issued, shares_repurchased, total_debt, terminal_gr, non_cash_change_working_capital, DnA, cap_ex, after_tax_operating_profit, company_name, market_index, domestic_revenue_percentage, sector_domestic_revenue_percentage, debt, equity, 
    implied_equity_risk_premium_sp500, bond_market_stddev, tax_rate, longest_gov_bond_rate, default_spread, other_revenue_country, other_default_spread, company_default_spread) = get_input_data()

    stock_returns, market_returns = fetch_stock_and_market_returns(company_name, market_index)
    beta = calculate_beta(stock_returns, market_returns)
    risk_free_rate = longest_gov_bond_rate - default_spread
    country_equity_risk_premium = implied_equity_risk_premium_sp500 + default_spread * (np.std(market_returns) / bond_market_stddev)
    company_equity_risk_premium = risk_free_rate + country_equity_risk_premium + (domestic_revenue_percentage / sector_domestic_revenue_percentage)
    cost_of_equity = risk_free_rate + beta * (1 + (1 - tax_rate) * (debt / equity)) * (company_equity_risk_premium - risk_free_rate)
    cost_of_debt = risk_free_rate + (domestic_revenue_percentage / 100) * default_spread + company_default_spread
    if other_revenue_country > 0:
        cost_of_debt += (other_revenue_country / 100) * other_default_spread
    wacc = (cost_of_equity * (equity / (equity + debt))) + (cost_of_debt * (1 - tax_rate) * (debt / (debt + equity)))
    
    FCFF = after_tax_operating_profit - (cap_ex - DnA) - non_cash_change_working_capital
    EV = FCFF/(wacc-terminal_gr)
    Equity_Value = EV - (total_debt - cash)
    Equity_Value_Per_Share = Equity_Value/(shares_issued - shares_repurchased)
    
    
    print(f"Calculated WACC: {wacc:.4f}")
    print(f"Cost of Equity: {cost_of_equity:.4f}")
    print(f"Cost of Debt: {cost_of_debt:.4f}")
    print(f"Beta: {beta:.4f}")
    print(f"Enterprise Value: {EV:.4f}")
    print(f"Equity Value: {Equity_Value:.4f}")
    print(f"Equity Value Per Share: {Equity_Value_Per_Share:.4f}")

In [ ]:
dcf_calculate()